In [89]:
import pymysql
from datetime import datetime, date,timezone,timedelta
import time 

from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.ie.options import Options
from selenium.webdriver.support.ui import Select,WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

host = '10.55.23.101'
port = 33060
user = 'root'
passwd = "1234"
db = 'logistic'

In [88]:
conn = pymysql.connect(host=host, port=port, user=user, passwd=passwd, db=db)
cur = conn.cursor()

0

In [2]:
url = 'http://163.29.117.215/khbweb/ShipinP.aspx'
option = webdriver.ChromeOptions()
driver = webdriver.Chrome(r"D:\chromedriver.exe",chrome_options=option)
driver.get(url)

C:\Users\jacob.liang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


In [80]:
cur.close()
conn.close()

In [102]:
conn = pymysql.connect(host=host, port=port, user=user, passwd=passwd, db=db)
cur = conn.cursor()

# 第 1 頁共 18 頁 (共 359 筆)
totalPage = int(driver.find_elements_by_class_name("dxp-summary")[0].text.split('共')[1].split('頁')[0].strip())
#totalPage = 4

cur.execute('delete from logistic.khbweb') 
cur.execute('ALTER TABLE logistic.khbweb AUTO_INCREMENT = 1')
for p in range(0,totalPage) :
    if p > 0 :
        driver.execute_script("ASPx.GVPagerOnClick('ASPx_預定進港','PN"+str(p)+"');")
        time.sleep(10)
    print('page:',p)    
    for r in range(0,20) :
        #print(p,r,c,cols)
        print(p+1,'/',totalPage,' row:',r)
        # 0:'shipno,voyage'
        data = driver.find_element_by_xpath('//*[@id="ASPx_預定進港_tccell'+str(r+p*20)+'_0"]').text
        shipno = data.split('\n')[0]
        voyage = data.split('\n')[1]
        print(' 0. shipno:',shipno,',voyage:',voyage)
        
        # 1:'berthing'
        berthing = None
        data = driver.find_element_by_xpath('//*[@id="ASPx_預定進港_tccell'+str(r+p*20)+'_1"]').text
        if data != '' :
            berthing = driver.find_element_by_xpath('//*[@id="ASPx_預定進港_tccell'+str(r+p*20)+'_1"]').text
        
        print(' 1. berthing:',berthing)
        
        # 2:'shipname_tw,shipname_en'
        data = driver.find_element_by_xpath('//*[@id="ASPx_預定進港_tccell'+str(r+p*20)+'_2"]').text
        shipname_tw = data.split('\n')[0]
        shipname_en = data.split('\n')[1]
        print(' 2. shipname_tw:',shipname_tw,'shipname_en:',shipname_en)
        
        # 3:'length_m,tonnage,agent_no,agent_name', 
        data = driver.find_element_by_xpath('//*[@id="ASPx_預定進港_tccell'+str(r+p*20)+'_3"]').text
        length_m = data.split('\n')[0].split('/')[0].replace('m', '')
        tonnage = data.split('\n')[0].split('/')[1]
        agent_no = data.split('\n')[1].split('-')[0]
        agent_name = data.split('\n')[1].split('-')[1]
        print(' 3. length_m:',length_m,',tonnage:',tonnage,',agent_no:',agent_no,',agent_name:',agent_name)
        
        # 4:'pilot_apply_datetime,pilot_apply_YC,check_datetime,check_YC'
        data = driver.find_element_by_xpath('//*[@id="ASPx_預定進港_tccell'+str(r+p*20)+'_4"]').text
        pilot_apply_datetime = None
        pilot_apply_YC = None
        if len(data.split('\n')[0].split('/')) == 4 :
            #pilot_apply_datetime = '/'.join(data.split('\n')[0].split('/')[0:3])
            pilot_apply_datetime = str(int(data.split('\n')[0].split('/')[0]) + 1911)+'/'+data.split('\n')[0].split('/')[1]+'/'+data.split('\n')[0].split('/')[2]
            pilot_apply_YC = data.split('\n')[0].split('/')[3]

        check_datetime = None
        check_YC = None    
        if len(data.split('\n')[1].split('/')) == 4 :
            #check_datetime = data.split('\n')[1].split('/')[0:3]
            check_datetime = str(int(data.split('\n')[1].split('/')[0]) + 1911)+'/'+data.split('\n')[1].split('/')[1]+'/'+data.split('\n')[1].split('/')[2]
            check_YC = data.split('\n')[1].split('/')[3]
        print(' 4. pilot_apply_datetime:',pilot_apply_datetime,',pilot_apply_YC:',pilot_apply_YC,',check_datetime:',check_datetime,',check_YC:',check_YC)

        # 6:'scheduled_arrival_datetime,captain_eta',
        data = driver.find_element_by_xpath('//*[@id="ASPx_預定進港_tccell'+str(r+p*20)+'_6"]').text
        scheduled_arrival_datetime = None
        captain_eta = None
        #data = driver.find_element_by_xpath('//*[@id="ASPx_預定進港_tccell3_6"]').text
        if len(data.split('\n')) == 2 :
            #scheduled_arrival_datetime = data.split('\n')[0]
            scheduled_arrival_datetime = str(int(data.split('\n')[0].split('/')[0]) + 1911)+'/'+data.split('\n')[0].split('/')[1]+'/'+data.split('\n')[0].split('/')[2]
            #captain_eta = data.split('\n')[1]
            captain_eta = str(int(data.split('\n')[1].split('/')[0]) + 1911)+'/'+data.split('\n')[1].split('/')[1]+'/'+data.split('\n')[1].split('/')[2]
        elif data != '' :
            scheduled_arrival_datetime = str(int(data.split('/')[0]) + 1911)+'/'+data.split('/')[1]+'/'+data.split('/')[2]
        print(' 6. scheduled_arrival_datetime:',scheduled_arrival_datetime,',captain_eta:',captain_eta)
        
        # 7:'actually_arrived_datetime,anchor_datetime'
        data = driver.find_element_by_xpath('//*[@id="ASPx_預定進港_tccell'+str(r+p*20)+'_7"]').text
        actually_arrived_datetime = None
        anchor_datetime = None
        if len(data.split('\n')) == 2 :
            actually_arrived_datetime = str(int(data.split('\n')[0].split('/')[0]) + 1911)+'/'+data.split('\n')[0].split('/')[1]+'/'+data.split('\n')[0].split('/')[2]
            anchor_datetime = str(int(data.split('\n')[1].split('/')[0]) + 1911)+'/'+data.split('\n')[1].split('/')[1]+'/'+data.split('\n')[1].split('/')[2]
        elif len(data.split('\n')) == 1 :
            if data != '' :
                actually_arrived_datetime = str(int(data.split('/')[0]) + 1911)+'/'+data.split('/')[1]+'/'+data.split('/')[2]
        print(' 7. actually_arrived_datetime:',actually_arrived_datetime,',anchor_datetime:',anchor_datetime)
        
        # 8:'harbor,action'
        data = driver.find_element_by_xpath('//*[@id="ASPx_預定進港_tccell'+str(r+p*20)+'_8"]').text
        harbor = None
        action = None
        harbor = data.split('\n')[0]
        action = data.split('\n')[1]
        print(' 8. harbor:',harbor,',action:',action)
        
        # 9:'shiptype,destination'
        data = driver.find_element_by_xpath('//*[@id="ASPx_預定進港_tccell'+str(r+p*20)+'_9"]').text
        shiptype = None
        destination = None
        if len(data.split('\n')) == 2 :    
            shiptype = data.split('\n')[0]
            destination = data.split('\n')[1]
        else :
            shiptype = data            
        print(' 9. shiptype:',shiptype,',destination:',destination)
        if shiptype != '全貨櫃' :
            continue 
        
        # 10:'imo,rescue,issc_csr'    
        data = driver.find_element_by_xpath('//*[@id="ASPx_預定進港_tccell'+str(r+p*20)+'_10"]').text
        imo = None
        rescue = None
        issc_csr = None
        #data = driver.find_element_by_xpath('//*[@id="ASPx_預定進港_tccell12_10"]').text
        if len(data.split('\n')) == 2 :   
            if data.split('\n')[0] != '' :
                imo = data.split('\n')[0]
            if data.split('\n')[1].split('/')[0] != '' :
                rescue = data.split('\n')[1].split('/')[0]
            if data.split('\n')[1].split('/')[1] != '' :
                issc_csr = data.split('\n')[1].split('/')[1]
        elif len(data.split('\n')) == 1 :   
            if '/' in data :
                if data.split('/')[0] != '' :
                    rescue = data.split('/')[0]
                if data.split('/')[1] != '' :
                    issc_csr = data.split('/')[1]
            elif data != '' :
                imo = data
        print(' 10. imo:',imo,',rescue:',rescue,',issc_csr:',issc_csr)    
        cur.execute('insert into logistic.khbweb (shipno,voyage,berthing,shipname_tw,shipname_en,length_m,tonnage,agent_no,agent_name,pilot_apply_datetime,pilot_apply_YC,check_datetime,check_YC,scheduled_arrival_datetime,captain_eta,actually_arrived_datetime,anchor_datetime,harbor,action,shiptype,destination,imo,rescue,issc_csr)values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)',(shipno,voyage,berthing,shipname_tw,shipname_en,length_m,tonnage,agent_no,agent_name,pilot_apply_datetime,pilot_apply_YC,check_datetime,check_YC,scheduled_arrival_datetime,captain_eta,actually_arrived_datetime,anchor_datetime,harbor,action,shiptype,destination,imo,rescue,issc_csr))
        cur.execute('commit') 
cur.execute('commit')    
cur.close()
conn.close()    

page: 0
1 / 17  row: 0
 0. shipno: 527122 ,voyage: 0007
 1. berthing: 8807
 2. shipname_tw: 忠正5號 shipname_en: CHUNG CHENG NO.5
 3. length_m: 16 ,tonnage: 39 ,agent_no: 1309 ,agent_name: 忠正興業
 4. pilot_apply_datetime: 2021/03/17 18:00 ,pilot_apply_YC: C ,check_datetime: None ,check_YC: None
 6. scheduled_arrival_datetime: 2021/03/17 18:00 ,captain_eta: None
 7. actually_arrived_datetime: None ,anchor_datetime: None
 8. harbor: 1 ,action: 空船
 9. shiptype: 拖船 ,destination: None
1 / 17  row: 1
 0. shipno: 323586 ,voyage: 0028
 1. berthing: 8601
 2. shipname_tw: 特立威 shipname_en: TRI WINGS
 3. length_m: 106 ,tonnage: 4014 ,agent_no: 2458 ,agent_name: 鼎勝
 4. pilot_apply_datetime: None ,pilot_apply_YC: None ,check_datetime: None ,check_YC: None
 6. scheduled_arrival_datetime: 2021/03/23 08:00 ,captain_eta: None
 7. actually_arrived_datetime: None ,anchor_datetime: None
 8. harbor: 1 ,action: 補給
 9. shiptype: 油輪 ,destination: None
1 / 17  row: 2
 0. shipno: 310069 ,voyage: 0013
 1. berthing: No

 9. shiptype: 油輪 ,destination: None
1 / 17  row: 18
 0. shipno: 465658 ,voyage: 0086
 1. berthing: None
 2. shipname_tw: 冠軍輪 shipname_en: SALVAGE CHAMPION
 3. length_m: 65 ,tonnage: 1830 ,agent_no: 1139 ,agent_name: 亞洲
 4. pilot_apply_datetime: None ,pilot_apply_YC: None ,check_datetime: None ,check_YC: None
 6. scheduled_arrival_datetime: 2021/03/31 13:00 ,captain_eta: None
 7. actually_arrived_datetime: None ,anchor_datetime: None
 8. harbor: 1 ,action: 補給
 9. shiptype: 拖船兼救難 ,destination: None
1 / 17  row: 19
 0. shipno: 560375 ,voyage: 0010
 1. berthing: 8601
 2. shipname_tw: 勝利 shipname_en: SHENG LI
 3. length_m: 54 ,tonnage: 495 ,agent_no: 1360 ,agent_name: 君橡船務
 4. pilot_apply_datetime: None ,pilot_apply_YC: None ,check_datetime: None ,check_YC: None
 6. scheduled_arrival_datetime: 2021/03/31 15:00 ,captain_eta: None
 7. actually_arrived_datetime: None ,anchor_datetime: None
 8. harbor: 0 ,action: 補給
 9. shiptype: 冷凍船 ,destination: None
page: 1
2 / 17  row: 0
 0. shipno: 562998 

 10. imo: 9117181 ,rescue: 1 ,issc_csr: Y
2 / 17  row: 16
 0. shipno: 051993 ,voyage: 0063
 1. berthing: 1070
 2. shipname_tw: 德翔都伯格 shipname_en: HANSA DUBURG
 3. length_m: 175 ,tonnage: 18296 ,agent_no: 1254 ,agent_name: 龍忻船務
 4. pilot_apply_datetime: 2021/04/06 18:30 ,pilot_apply_YC: Y ,check_datetime: 2021/04/06 20:30 ,check_YC: __
 6. scheduled_arrival_datetime: 2021/04/05 22:00 ,captain_eta: None
 7. actually_arrived_datetime: None ,anchor_datetime: None
 8. harbor: 2 ,action: 裝卸貨
 9. shiptype: 全貨櫃 ,destination: None
 10. imo: 9535096 ,rescue: 1 ,issc_csr: Y
2 / 17  row: 17
 0. shipno: 065323 ,voyage: 0047
 1. berthing: 1076
 2. shipname_tw: 華航6 shipname_en: HUA HANG 6
 3. length_m: 130 ,tonnage: 7545 ,agent_no: 5862 ,agent_name: 昌和船務代理
 4. pilot_apply_datetime: 2021/04/07 16:00 ,pilot_apply_YC: Y ,check_datetime: None ,check_YC: None
 6. scheduled_arrival_datetime: 2021/04/05 23:00 ,captain_eta: None
 7. actually_arrived_datetime: None ,anchor_datetime: None
 8. harbor: 2 ,action

 9. shiptype: 全貨櫃 ,destination: 澳洲
 10. imo: 9231250 ,rescue: 1 ,issc_csr: Y
3 / 17  row: 12
 0. shipno: 553204 ,voyage: 0018
 1. berthing: 8601
 2. shipname_tw: 麥拉倫 shipname_en: MAI LA LUN
 3. length_m: 75 ,tonnage: 1497 ,agent_no: 1360 ,agent_name: 君橡船務
 4. pilot_apply_datetime: None ,pilot_apply_YC: None ,check_datetime: None ,check_YC: None
 6. scheduled_arrival_datetime: 2021/04/06 15:00 ,captain_eta: 2021/04/06 12:30
 7. actually_arrived_datetime: None ,anchor_datetime: None
 8. harbor: 0 ,action: 修護
 9. shiptype: 雜貨船 ,destination: None
3 / 17  row: 13
 0. shipno: 948225 ,voyage: 0023
 1. berthing: 1057
 2. shipname_tw: 阿羅漢 shipname_en: ARAHAN
 3. length_m: 109 ,tonnage: 5173 ,agent_no: 1014 ,agent_name: 三光公司
 4. pilot_apply_datetime: 2021/04/07 11:30 ,pilot_apply_YC: Y ,check_datetime: 2021/04/07 13:30 ,check_YC: __
 6. scheduled_arrival_datetime: 2021/04/06 15:00 ,captain_eta: None
 7. actually_arrived_datetime: None ,anchor_datetime: None
 8. harbor: 1 ,action: 卸非兩岸
 9. shipty

 9. shiptype: 多用途船 ,destination: 兩岸
4 / 17  row: 9
 0. shipno: 509558 ,voyage: 0015
 1. berthing: 1054
 2. shipname_tw: 新界 shipname_en: NEW BOUNDARY
 3. length_m: 190 ,tonnage: 31878 ,agent_no: 1373 ,agent_name: 榮華船務
 4. pilot_apply_datetime: 2021/04/07 04:30 ,pilot_apply_YC: Y ,check_datetime: 2021/04/07 06:00 ,check_YC: __
 6. scheduled_arrival_datetime: 2021/04/06 23:00 ,captain_eta: 2021/04/03 19:30
 7. actually_arrived_datetime: 2021/04/03 19:28 ,anchor_datetime: 2021/04/03 19:54
 8. harbor: 1 ,action: 卸非兩岸
 9. shiptype: 散裝船 ,destination: None
4 / 17  row: 10
 0. shipno: 036757 ,voyage: 0197
 1. berthing: 1070
 2. shipname_tw: 暢明 shipname_en: YM INITIATIVE
 3. length_m: 163 ,tonnage: 16488 ,agent_no: 1331 ,agent_name: 陽明海運
 4. pilot_apply_datetime: 2021/04/07 11:00 ,pilot_apply_YC: Y ,check_datetime: 2021/04/07 12:00 ,check_YC: __
 6. scheduled_arrival_datetime: 2021/04/07 01:00 ,captain_eta: None
 7. actually_arrived_datetime: None ,anchor_datetime: None
 8. harbor: 2 ,action: 裝卸

...

 7. actually_arrived_datetime: None ,anchor_datetime: None
 8. harbor: 2 ,action: 裝卸貨
 9. shiptype: 全貨櫃 ,destination: 非洲
 10. imo: 9525481 ,rescue: 1 ,issc_csr: Y
14 / 17  row: 12
 0. shipno: 035319 ,voyage: 0122
 1. berthing: 1108
 2. shipname_tw: 瑪格娜 shipname_en: MAGNA
 3. length_m: 109 ,tonnage: 5403 ,agent_no: 1395 ,agent_name: 驥達高分
 4. pilot_apply_datetime: 2021/04/13 08:00 ,pilot_apply_YC: Y ,check_datetime: None ,check_YC: None
 6. scheduled_arrival_datetime: 2021/04/13 08:00 ,captain_eta: None
 7. actually_arrived_datetime: None ,anchor_datetime: None
 8. harbor: 2 ,action: 裝卸貨
 9. shiptype: 全貨櫃 ,destination: None
 10. imo: 9296468 ,rescue: 1 ,issc_csr: Y
14 / 17  row: 13
 0. shipno: 045993 ,voyage: 0026
 1. berthing: 1065
 2. shipname_tw: 東方維拉 shipname_en: VELA
 3. length_m: 265 ,tonnage: 41225 ,agent_no: 2396 ,agent_name: 台灣東方海外
 4. pilot_apply_datetime: None ,pilot_apply_YC: None ,check_datetime: None ,check_YC: None
 6. scheduled_arrival_datetime: 2021/04/13 08:30 ,captain_

 2. shipname_tw: 貝爾 shipname_en: BERGE BANDAI
 3. length_m: 180 ,tonnage: 24872 ,agent_no: 1343 ,agent_name: 三強公司
 4. pilot_apply_datetime: None ,pilot_apply_YC: None ,check_datetime: None ,check_YC: None
 6. scheduled_arrival_datetime: 2021/04/15 12:00 ,captain_eta: None
 7. actually_arrived_datetime: None ,anchor_datetime: None
 8. harbor: 2 ,action: 卸貨
 9. shiptype: 散裝船 ,destination: None
15 / 17  row: 9
 0. shipno: 061034 ,voyage: 0084
 1. berthing: 1079
 2. shipname_tw: 長源 shipname_en: EVER BIRTH
 3. length_m: 212 ,tonnage: 33266 ,agent_no: 5470 ,agent_name: 長榮國際公司
 4. pilot_apply_datetime: None ,pilot_apply_YC: None ,check_datetime: None ,check_YC: None
 6. scheduled_arrival_datetime: 2021/04/15 12:00 ,captain_eta: None
 7. actually_arrived_datetime: None ,anchor_datetime: None
 8. harbor: 2 ,action: 裝卸貨
 9. shiptype: 全貨櫃 ,destination: None
 10. imo: 9786956 ,rescue: 1 ,issc_csr: Y
15 / 17  row: 10
 0. shipno: 064343 ,voyage: 0035
 1. berthing: 1079
 2. shipname_tw: 長晶 shipname_e

 6. scheduled_arrival_datetime: 2021/04/16 12:00 ,captain_eta: None
 7. actually_arrived_datetime: None ,anchor_datetime: None
 8. harbor: 2 ,action: 裝卸貨
 9. shiptype: 全貨櫃 ,destination: 兩岸
 10. imo: 9202235 ,rescue: 1 ,issc_csr: Y
16 / 17  row: 5
 0. shipno: 061824 ,voyage: 0080
 1. berthing: 1116
 2. shipname_tw: 吉邦 shipname_en: SINAR SUMBA
 3. length_m: 176 ,tonnage: 18321 ,agent_no: 5470 ,agent_name: 長榮國際公司
 4. pilot_apply_datetime: None ,pilot_apply_YC: None ,check_datetime: None ,check_YC: None
 6. scheduled_arrival_datetime: 2021/04/16 12:00 ,captain_eta: None
 7. actually_arrived_datetime: None ,anchor_datetime: None
 8. harbor: 2 ,action: 裝卸貨
 9. shiptype: 全貨櫃 ,destination: 東南
 10. imo: 9435222 ,rescue: 1 ,issc_csr: Y
16 / 17  row: 6
 0. shipno: 043184 ,voyage: 0283
 1. berthing: 1065
 2. shipname_tw: 騰雲河 shipname_en: TENG YUN HE
 3. length_m: 180 ,tonnage: 20569 ,agent_no: 2396 ,agent_name: 台灣東方海外
 4. pilot_apply_datetime: None ,pilot_apply_YC: None ,check_datetime: None ,chec

 0. shipno: 000612 ,voyage: 0139
 1. berthing: 1080
 2. shipname_tw: 立美 shipname_en: UNI-PERFECT
 3. length_m: 182 ,tonnage: 17887 ,agent_no: 5470 ,agent_name: 長榮國際公司
 4. pilot_apply_datetime: None ,pilot_apply_YC: None ,check_datetime: None ,check_YC: None
 6. scheduled_arrival_datetime: 2021/04/19 12:00 ,captain_eta: None
 7. actually_arrived_datetime: None ,anchor_datetime: None
 8. harbor: 2 ,action: 裝卸貨
 9. shiptype: 全貨櫃 ,destination: 東南
 10. imo: 9202182 ,rescue: 1 ,issc_csr: Y
17 / 17  row: 2
 0. shipno: 063983 ,voyage: 0047
 1. berthing: 1081
 2. shipname_tw: 德塔 shipname_en: TOKYO TOWER
 3. length_m: 172 ,tonnage: 17229 ,agent_no: 5470 ,agent_name: 長榮國際公司
 4. pilot_apply_datetime: None ,pilot_apply_YC: None ,check_datetime: None ,check_YC: None
 6. scheduled_arrival_datetime: 2021/04/19 12:00 ,captain_eta: None
 7. actually_arrived_datetime: None ,anchor_datetime: None
 8. harbor: 2 ,action: 裝卸貨
 9. shiptype: 全貨櫃 ,destination: 東南
 10. imo: 9384875 ,rescue: 1 ,issc_csr: Y
17 / 1

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="ASPx_預定進港_tccell336_0"]"}
  (Session info: chrome=85.0.4183.83)
  (Driver info: chromedriver=2.43.600210 (68dcf5eebde37173d4027fa8635e332711d2874a),platform=Windows NT 10.0.17134 x86_64)


In [78]:
data.split('\n')[0].split('/')[0]

'110'

In [3]:
collist = {0:'shipno,voyage',1:'berthing',2:'shipname_tw,shipname_en',3:'length_m,tonnage,agent_no,agent_name', 
4:'pilot_apply_datetime,pilot_apply_YC,check_datetime,check_YC',6:'scheduled_arrival_datetime,captain_eta',
7:'actually_arrived_datetime,anchor_datetime',8:'port,action',9:'shiptype,destination',10:'imo,rescue,issc_csr'}

In [53]:
# imo,rescue,issc_csr
imo = None
rescue = None
issc_csr = None
data = driver.find_element_by_xpath('//*[@id="ASPx_預定進港_tccell12_10"]').text
if len(data.split('\n')) == 2 :   
    if data.split('\n')[0] != '' :
        imo = data.split('\n')[0]
    if data.split('\n')[1].split('/')[0] != '' :
        rescue = data.split('\n')[1].split('/')[0]
    if data.split('\n')[1].split('/')[1] != '' :
        issc_csr = data.split('\n')[1].split('/')[1]
elif len(data.split('\n')) == 1 :   
    if '/' in data :
        if data.split('/')[0] != '' :
            rescue = data.split('/')[0]
        if data.split('/')[1] != '' :
            issc_csr = data.split('/')[1]
    elif data != '' :
        imo = data
print('imo:',imo,',rescue:',rescue,',issc_csr:',issc_csr)


imo: None ,rescue: None ,issc_csr: None


In [43]:
data

'1/Y'

In [49]:
len(driver.find_element_by_xpath('//*[@id="ASPx_預定進港_tccell0_10"]').text.split('\n'))

1

In [15]:
data=''
len(data.split('\n'))
#data

1

In [7]:
data.split('\n')[0].split('/')[3]

'C'

In [20]:
# pilot_apply_datetime,pilot_apply_YC,check_datetime,check_YC
pilot_apply_datetime = ''
pilot_apply_YC = ''
if len(data.split('\n')[0].split('/')) == 4 :
    pilot_apply_datetime = data.split('\n')[0].split('/')[0:2]
    pilot_apply_YC = data.split('\n')[0].split('/')[3]
    
check_datetime = ''
check_YC = ''    
if len(data.split('\n')[1].split('/')) == 4 :
    check_datetime = data.split('\n')[1].split('/')[0:2]
    check_YC = data.split('\n')[1].split('/')[3]
    

2

In [22]:
data.split('\n')[1].split('/')

'__'

In [ ]:
# 第 1 頁共 18 頁 (共 359 筆)
totalPage = int(driver.find_elements_by_class_name("dxp-summary")[0].text.split('共')[1].split('頁')[0].strip())
totalPage = 1

for p in range(0,totalPage) :
    if p > 0 :
        driver.execute_script("ASPx.GVPagerOnClick('ASPx_預定進港','PN"+str(p)+"');")
        time.sleep(4)
    for r in range(0,20) :
        for c,cols in collist.items():
            print(p,r,c,cols)
            data = driver.find_element_by_xpath('//*[@id="ASPx_預定進港_tccell'+str(r+p*20)+'_'+str(c)+'"]').text
            for col in cols.split(',') :
                print(col)
                
            
            print('  '+data+'\n')
    
    

0 0 0 shipno,voyage
  527122
0007

0 0 1 berthing
  8807

0 0 2 shipname_tw,shipname_en
  忠正5號
CHUNG CHENG NO.5

0 0 3 length_m,tonnage,agent_no,agent_name
  16m/39
1309-忠正興業

0 0 4 pilot_apply_datetime,pilot_apply_YC,check_datetime,check_YC
  110/03/17 18:00/C
/__

0 0 6 scheduled_arrival_datetime,captain_eta
  110/03/17 18:00

0 0 7 actually_arrived_datetime,anchor_datetime
  

0 0 8 port,action
  1
空船

0 0 9 shiptype,destination
  拖船

0 0 10 imo,rescue,issc_csr
  

0 1 0 shipno,voyage
  529443
0025

0 1 1 berthing
  8601

0 1 2 shipname_tw,shipname_en
  鉅星
MAGIC STAR

0 1 3 length_m,tonnage,agent_no,agent_name
  100m/4240
9163-百利船務代理

0 1 4 pilot_apply_datetime,pilot_apply_YC,check_datetime,check_YC
  /
/__

0 1 6 scheduled_arrival_datetime,captain_eta
  110/03/20 08:00

0 1 7 actually_arrived_datetime,anchor_datetime
  

0 1 8 port,action
  0
補給

0 1 9 shiptype,destination
  油輪

0 1 10 imo,rescue,issc_csr
  

0 2 0 shipno,voyage
  532312
0017

0 2 1 berthing
  

0 2 2 shipname_tw,s

0 18 0 shipno,voyage
  290955
0001

0 18 1 berthing
  1008

0 18 2 shipname_tw,shipname_en
  Angelinna
Angelinna

0 18 3 length_m,tonnage,agent_no,agent_name
  13m/34
8717-

0 18 4 pilot_apply_datetime,pilot_apply_YC,check_datetime,check_YC
  110/03/29 12:00/C
/__

0 18 6 scheduled_arrival_datetime,captain_eta
  110/03/29 12:00

0 18 7 actually_arrived_datetime,anchor_datetime
  

0 18 8 port,action
  1
補給

0 18 9 shiptype,destination
  自用遊艇

0 18 10 imo,rescue,issc_csr
  

0 19 0 shipno,voyage
  554715
0018

0 19 1 berthing
  8601

0 19 2 shipname_tw,shipname_en
  順發1號
SHUN FA NO.1

0 19 3 length_m,tonnage,agent_no,agent_name
  75m/1025
9163-百利船務代理

0 19 4 pilot_apply_datetime,pilot_apply_YC,check_datetime,check_YC
  /
/__

0 19 6 scheduled_arrival_datetime,captain_eta
  110/03/30 08:00

0 19 7 actually_arrived_datetime,anchor_datetime
  

0 19 8 port,action
  0
補給

0 19 9 shiptype,destination
  油輪

0 19 10 imo,rescue,issc_csr
  

1 0 0 shipno,voyage
  413909
0002

1 0 1 berthing
  



  1
補給

1 15 9 shiptype,destination
  工作船

1 15 10 imo,rescue,issc_csr
  

1 16 0 shipno,voyage
  044512
0086

1 16 1 berthing
  1069

1 16 2 shipname_tw,shipname_en
  現代恩典
HYUNDAI GRACE

1 16 3 length_m,tonnage,agent_no,agent_name
  294m/52581
1800-台灣韓新遠洋

1 16 4 pilot_apply_datetime,pilot_apply_YC,check_datetime,check_YC
  110/04/04 04:30/Y
110/04/04 06:00/YY

1 16 6 scheduled_arrival_datetime,captain_eta
  110/04/02 08:00
110/04/04 05:30

1 16 7 actually_arrived_datetime,anchor_datetime
  

1 16 8 port,action
  2
裝卸貨

1 16 9 shiptype,destination
  全貨櫃
東南

1 16 10 imo,rescue,issc_csr
  

1 17 0 shipno,voyage
  387436
0059

1 17 1 berthing
  1028

1 17 2 shipname_tw,shipname_en
  豐帆27
FENG FAN 27

1 17 3 length_m,tonnage,agent_no,agent_name
  115m/4682
1212-陞暘船務

1 17 4 pilot_apply_datetime,pilot_apply_YC,check_datetime,check_YC
  110/04/04 10:00/Y
110/04/04 11:30/__

1 17 6 scheduled_arrival_datetime,captain_eta
  110/04/02 12:00

1 17 7 actually_arrived_datetime,anchor_datetime
  



  2
裝卸非兩

2 12 9 shiptype,destination
  全貨櫃
兩岸

2 12 10 imo,rescue,issc_csr
  

2 13 0 shipno,voyage
  065943
0005

2 13 1 berthing
  1110

2 13 2 shipname_tw,shipname_en
  馬希拉布快輪
AL MASHRAB

2 13 3 length_m,tonnage,agent_no,agent_name
  369m/153148
1382-台灣赫伯羅德

2 13 4 pilot_apply_datetime,pilot_apply_YC,check_datetime,check_YC
  110/04/04 08:00/Y
110/04/04 09:30/__

2 13 6 scheduled_arrival_datetime,captain_eta
  110/04/03 19:00

2 13 7 actually_arrived_datetime,anchor_datetime
  

2 13 8 port,action
  2
裝卸貨

2 13 9 shiptype,destination
  全貨櫃
歐洲

2 13 10 imo,rescue,issc_csr
  

2 14 0 shipno,voyage
  045993
0025

2 14 1 berthing
  1065

2 14 2 shipname_tw,shipname_en
  東方維拉
VELA

2 14 3 length_m,tonnage,agent_no,agent_name
  265m/41225
2396-台灣東方海外

2 14 4 pilot_apply_datetime,pilot_apply_YC,check_datetime,check_YC
  110/04/04 15:30/Y
110/04/04 17:30/__

2 14 6 scheduled_arrival_datetime,captain_eta
  110/04/03 21:00

2 14 7 actually_arrived_datetime,anchor_datetime
  

2 14 8 port,act

  8901

3 10 2 shipname_tw,shipname_en
  東方68號
DF NO.68

3 10 3 length_m,tonnage,agent_no,agent_name
  73m/2306
1495-正華船務代理

3 10 4 pilot_apply_datetime,pilot_apply_YC,check_datetime,check_YC
  110/04/04 07:30/C
110/04/04 09:00/__

3 10 6 scheduled_arrival_datetime,captain_eta
  110/04/04 08:00
110/04/03 07:50

3 10 7 actually_arrived_datetime,anchor_datetime
  

3 10 8 port,action
  2
補給

3 10 9 shiptype,destination
  工作平台船

3 10 10 imo,rescue,issc_csr
  

3 11 0 shipno,voyage
  567019
0003

3 11 1 berthing
  8901

3 11 2 shipname_tw,shipname_en
  金興2801
KIM HENG 2801

3 11 3 length_m,tonnage,agent_no,agent_name
  38m/445
1495-正華船務代理

3 11 4 pilot_apply_datetime,pilot_apply_YC,check_datetime,check_YC
  110/04/04 07:30/C
110/04/04 09:00/__

3 11 6 scheduled_arrival_datetime,captain_eta
  110/04/04 08:00
110/04/03 07:50

3 11 7 actually_arrived_datetime,anchor_datetime
  110/04/03 07:31

3 11 8 port,action
  1
補給

3 11 9 shiptype,destination
  拖船

3 11 10 imo,rescue,issc_csr
  

3 12 0 

  

4 7 0 shipno,voyage
  065676
0017

4 7 1 berthing
  1070

4 7 2 shipname_tw,shipname_en
  碩明
YM CELEBRITY

4 7 3 length_m,tonnage,agent_no,agent_name
  210m/32720
1331-陽明海運

4 7 4 pilot_apply_datetime,pilot_apply_YC,check_datetime,check_YC
  110/04/04 20:00/Y
110/04/04 21:00/__

4 7 6 scheduled_arrival_datetime,captain_eta
  110/04/04 20:00

4 7 7 actually_arrived_datetime,anchor_datetime
  

4 7 8 port,action
  2
裝卸貨

4 7 9 shiptype,destination
  全貨櫃

4 7 10 imo,rescue,issc_csr
  

4 8 0 shipno,voyage
  557919
0038

4 8 1 berthing
  1017

4 8 2 shipname_tw,shipname_en
  大山
TA SHAN

4 8 3 length_m,tonnage,agent_no,agent_name
  80m/1673
2042-大山航運公司

4 8 4 pilot_apply_datetime,pilot_apply_YC,check_datetime,check_YC
  /
110/04/04 22:30/__

4 8 6 scheduled_arrival_datetime,captain_eta
  110/04/04 22:00

4 8 7 actually_arrived_datetime,anchor_datetime
  

4 8 8 port,action
  1
裝卸貨

4 8 9 shiptype,destination
  雜貨船

4 8 10 imo,rescue,issc_csr
  

4 9 0 shipno,voyage
  042854
0322

4 9 1 


5 4 3 length_m,tonnage,agent_no,agent_name
  107m/6094
9163-百利船務代理

5 4 4 pilot_apply_datetime,pilot_apply_YC,check_datetime,check_YC
  110/04/05 21:00/Y
/__

5 4 6 scheduled_arrival_datetime,captain_eta
  110/04/05 07:00

5 4 7 actually_arrived_datetime,anchor_datetime
  

5 4 8 port,action
  2
裝卸非兩

5 4 9 shiptype,destination
  全貨櫃
兩岸

5 4 10 imo,rescue,issc_csr
  

5 5 0 shipno,voyage
  315974
0643

5 5 1 berthing
  1079

5 5 2 shipname_tw,shipname_en
  騏龍
GODSPEED

5 5 3 length_m,tonnage,agent_no,agent_name
  117m/5272
5734-光華船務公司

5 5 4 pilot_apply_datetime,pilot_apply_YC,check_datetime,check_YC
  110/04/05 01:00/Y
110/04/05 06:00/__

5 5 6 scheduled_arrival_datetime,captain_eta
  110/04/05 07:00

5 5 7 actually_arrived_datetime,anchor_datetime
  

5 5 8 port,action
  2
裝卸非兩

5 5 9 shiptype,destination
  雜貨船
兩岸

5 5 10 imo,rescue,issc_csr
  

5 6 0 shipno,voyage
  020818
1535

5 6 1 berthing
  1117

5 6 2 shipname_tw,shipname_en
  驪龍
NAGALEADER

5 6 3 length_m,tonnage,agent_no,ag

In [7]:
cols='shipno'
len(cols.split(','))
for col in cols.split(',') :
    print(col)


shipno


In [ ]:
## ASPx_預定進港_tccell0_0
# 1.shipno, 船編
# 2.voyage, 航次

## ASPx_預定進港_tccell0_1
# 3.berthing, 靠泊碼頭

## ASPx_預定進港_tccell0_2
# 4.shipname_tw, 船名(中)
# 5.shipname_en, 船名(英)

## ASPx_預定進港_tccell0_3
# 6.length_m, 總長
# 7.tonnage, 噸位
# 8.agent_no, 港代理(編號)
# 9.agent_name, 港代理(名稱)

## ASPx_預定進港_tccell0_4
# 10.pilot_apply_datetime, 申請引水
# 11.pilot_apply_YC, 申請引水審核
# 12.check_datetime, 聯檢派檢
# 13.check_YC, 聯檢派檢審核	 

## ASPx_預定進港_tccell0_6
# 14.scheduled_arrival_datetime, 預定進港
# 15.captain_eta, 船長ETA	 

## ASPx_預定進港_tccell0_7
# 16.actually_arrived_datetime, 實際到達時間
# 17.anchor_datetime, 下錨時間

## ASPx_預定進港_tccell0_8
# 18.port, 進出港口
# 19.action, 目的

## ASPx_預定進港_tccell0_9
# 20.shiptype, 船舶種類
# 21.destination, 航線

## ASPx_預定進港_tccell0_10
# 22.imo, IMO
# 23.rescue, 保全
# 24.issc_csr, ISSC-CSR

In [8]:

# 1.shipno, 船編
shipno = driver.find_element_by_xpath("/html/body/form/table[2]/tbody/tr/td/table[1]/tbody/tr[3]/td[1]").text.split('\n')[0]

# 2.voyage, 航次
voyage = driver.find_element_by_xpath("/html/body/form/table[2]/tbody/tr/td/table[1]/tbody/tr[3]/td[1]").text.split('\n')[1]

print('shipno:',shipno,',voyage:',voyage)
# 3.berthing, 靠泊碼頭
//*[@id="ASPx_預定進港_tccell0_0"]
/html/body/form/table[2]/tbody/tr/td/table[1]/tbody/tr[3]/td[2]

# 4.shipname_tw, 船名(中)
# 5.shipname_en, 船名(英)
# 6.length_m, 總長
# 7.tonnage, 噸位
# 8.agent_no, 港代理(編號)
# 9.agent_name, 港代理(名稱)
# 10.pilot_apply_datetime, 申請引水
# 11.pilot_apply_YC, 申請引水審核
# 12.check_datetime, 聯檢派檢
# 13.check_YC, 聯檢派檢審核	 
# 14.scheduled_arrival_datetime, 預定進港
# 15.captain_eta, 船長ETA	 
# 16.actually_arrived_datetime, 實際到達時間
# 17.anchor_datetime, 下錨時間	 
# 18.port, 進出港口
# 19.action, 目的	 
# 20.shiptype, 船舶種類
# 21.destination, 航線	 
# 22.imo, IMO
# 23.rescue, 保全
# 24.issc_csr, ISSC-CSR



shipno: 527122 ,voyage: 0007


In [20]:
plist[2]

<tr>
<td style="vertical-align:Top;"><font size="5"><b><div class="dxlbd" id="ASPxComboBox_船名_DDD_L_D" style="overflow-x:hidden;overflow-y:auto;">
<input id="ASPxComboBox_船名_DDD_L_VI" name="ASPxComboBox_船名$DDD$L" type="hidden"/><table style="border-collapse:separate;visibility:hidden!important;display:none!important;">
<tr class="dxeListBoxItemRow" id="ASPxComboBox_船名_DDD_L_LBI-1">
<td class="dxeListBoxItem" id="ASPxComboBox_船名_DDD_L_LBII"><font size="5"><b> </b></font></td>
</tr>
</table><table id="ASPxComboBox_船名_DDD_L_LBT" style="border-collapse:separate;" width="100%">
</table>
</div></b></font></td>
</tr>

In [ ]:
//*[@id="ASPx_預定進港_DXDataRow0"]
/html/body/form/table[2]/tbody/tr/td/table[1]/tbody/tr[3]/td[1]

In [4]:
soup


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">
<head><link href="/khbweb/DXR.axd?r=1_284,1_281,1_66,1_67,1_68,1_283,1_280,1_72-f0Wkl" rel="stylesheet" type="text/css"/><link href="/khbweb/DXR.axd?r=1_71,0_4614,0_4737,0_4618-f0Wkl" rel="stylesheet" type="text/css"/><link href="/khbweb/DXR.axd?r=0_4741,0_4607,0_4611-c0Wkl" rel="stylesheet" type="text/css"/><meta content="width=device-width; initial-scale=1.0; maximum-scale=1.0; user-scalable=0;" name="viewport"/><title>
	預定進出移泊船舶
</title>
<style type="text/css">
        .style1 {
            font-size: medium;
            font-family: 微軟正黑體;
        }

        .style2 {
            font-family: 微軟正黑體;
        }
    </style>
</head>
<body bgcolor="#ffccff">
<form action="./ShipinP.aspx" id="form1" method="post" name="form1">
<input id="__VIEWSTATE" name="__VIEWSTATE" type="hidden" value="..."/>
<input id="__VIEWSTATEGEN